# Deriving the determining equations of the Lotka-Volterra model
*Date*: 2022-04-13,<br>
*Written by:* Johannes Borgqvist.<br>
Ok, so consider the dimensionless *Lotka-Volterra* model:
\begin{align}
    \dfrac{\mathrm{d}u}{\mathrm{d}t}&=u(1-v),\\
    \dfrac{\mathrm{d}v}{\mathrm{d}t}&=\alpha v(u-1).\\
\end{align}
Now, we would like to see if we can derive the determining equations from this using sympy. <br>

In [2]:
# Import sympy
from sympy import *
# Translate a string to symbolic expression
from sympy.parsing.sympy_parser import parse_expr
# Finding monomials
from sympy.polys.monomials import itermonomials, monomial_count
# Ordering monomials 
from sympy.polys.orderings import monomial_key
# For printing to a file
import sys

# Setting up the reaction terms and so on

In [3]:
# Independent variable
t = symbols('t')
# Dependent variables
u,v = symbols('u v',cls=Function)
# Define our parameter a
a = symbols('a')
# Define the states
states = [u(t), v(t)]
# Define our reaction terms
omega_list = [u(t)*(1-v(t)), a*v(t)*(u(t)-1)]

# Function for setting up the linearised symmetry condition
So now we will try to set up a general function for deriving the linearised symmetry conditions!

In [4]:
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# FUNCTION 1: "lin_sym_conds_fibre_preserving"
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# This function sets up the linearised symmetry conditions
# given an independent variable t, the states and the reaction terms
def lin_sym_conds_fibre_preserving(t,states,reaction_terms):
    # Temporary variables for the dependent variables so that we can 
    # differentiate the reaction terms with respect to each state
    x = symbols('x')    
    # Add a temporary function which we differentiate all the time
    f = symbols('f',cls=Function)
    # Define a list of tangents
    tangent_list = []
    # Define the time tangent
    xi = symbols('xi', cls=Function)
    # Add it to the tangent list
    tangent_list.append(xi(t))
    # Loop over the states and add a tangent for each state, hey?
    for state_index in range(1,len(states)+1):
        # Here, we essentially define a string which we then convert
        # to a symbolic expression using "parse_expr"
        temp_str = "eta_" + str(state_index) + "(" + str(t) + "," 
        for state in states:
            temp_str += str(state) + ","
        temp_str += ")"
        temp_str = temp_str.replace(",)",")")
        # Add our new tangent
        tangent_list.append(parse_expr(temp_str))
    # Ok, so at this point we have our tangent lists, now we can start assembling our linearised symmetry conditions
    # Allocate memory for our linearised symmetry conditions
    lin_sym_original = []
    # Loop over the number of states and define the linearised symmetry condition
    for state_index in range(1,len(states)+1):
        # Define the LHS
        LHS = Derivative(tangent_list[state_index],t).doit() - omega_list[state_index-1]*Derivative(tangent_list[0],t).doit()
        # Allocate memory for the RHS
        RHS = 0
        # Add all partial derivatives of the current reaction term by looping over all states and times
        for RHS_index in range(1,len(states)+1):
            # Extract temporary tangent
            tangent_temp = tangent_list[RHS_index]
            # Partial derivative wrt to the time
            RHS += (Derivative(omega_list[state_index-1].subs(states[RHS_index-1],x),x).subs(x,states[RHS_index-1]).doit()*tangent_temp)
        # After this is done we loop over the states one more time and substitute all time derivatives of the
        # states with the reaction terms
        eq_temp = LHS - RHS
        for state_index in range(len(omega_list)):
            # Also, we substitute the reaction terms in the LHS
            eq_temp = eq_temp.subs(Derivative(states[state_index],t),omega_list[state_index])
        # Append the equation defined as LHS-RHS
        lin_sym_original.append((eq_temp,0))
    # Return the list with all the tangents
    return tangent_list, lin_sym_original
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# FUNCTION 2: "latex_printing_of_lin_sym"
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# This function takes in the time t, the states, tangent_list and the linearised symmetry condition
# and prints them in a nice fashion.
def latex_printing_of_lin_sym(t,states,tangent_list,lin_sym_original):
    # We essentially return a giant string which can be printed, so let's start constructing it
    tangent_str = "\n\nThe tangents are:\n$$" + latex(tangent_list) + "$$\n"
    output_str = "\n\\begin{align*}\n"
    # Get the arguments as well
    argument_str = latex(tangent_list[1]).replace("\\eta_{1}","")
    # Loop through the linearised symmetry condition and add the equations
    for lin_sym in lin_sym_original:
        output_str += latex(lin_sym[0]) + "&=" + latex(lin_sym[1]) + ",\\\\\n"
    output_str += "\\end{align*}\n\n"
    # Now we replace the arguments as well to see if this helps
    output_str = output_str.replace(argument_str,"")  
    # Remove time dependence already
    output_str = output_str.replace("{\\left(t \\right)}","")    
    # Loop through the tangents and replace strings for partial derivatives
    for tangent_index in range(len(tangent_list)):
        if tangent_index == 0:
            # Time derivative
            time_str = "\\left. \\frac{\\partial}{\\partial \\xi_{1}} \\xi{\\left(\\xi_{1},u,v \\right)} \\right|_{\\substack{ \\xi_{1}=t }}"
            output_str = output_str.replace(time_str,"\\frac{d\\xi}{dt}")
            output_str = output_str.replace("\\frac{d}{d t} \\xi","\\frac{d\\xi}{dt}")
        else:
            # Time derivative
            time_str = "\\left. \\frac{\\partial}{\\partial \\xi_{1}} \\eta_{" + str(tangent_index) + "}{\\left(\\xi_{1},u,v \\right)} \\right|_{\\substack{ \\xi_{1}=t }}"
            output_str = output_str.replace(time_str,"\\frac{\\partial\\eta_"+ str(tangent_index)+"}{\\partial t}")            
            state_str = ""
            # Loop through the states and find the partial derivatives
            for state_index in range(len(states)):
                # Extract the string for the state at hand
                state_str = latex(states[state_index])
                state_str = state_str.replace("{\\left(t \\right)}","")
                # Replace the partial derivative at hand
                output_str = output_str.replace("\\frac{d}{d " + state_str +"} \\eta_{"+ str(tangent_index) +"}","\\frac{\\partial\\eta_{"+ str(tangent_index) +"}}{\\partial "+ state_str +"}")            
    # Return the output string
    return tangent_str,output_str
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# FUNCTION 3: "create_tangent_ansatze"
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# The function takes three inputs:
#1. The time variable t,
#2. The states in the list states, 
#2. The degree of the polynomial ("degree_polynomial").
# It returns two outputs:
# 1.The variables (independent and dependent) in the symmetry framework,
# 2. A list of all tangents ("eta_list").
# The function uses various functionalities in sympy and the polynomials are generated using the built in functions in the polys.monomial part of the sympy library. To automate the generation of the sympy functions and symbol, the "exec" command is used at multiple places in the function.
def create_tangent_ansatze(t,states,degree_polynomial):
    # Calculate the number of variables and number of states
    num_of_states = len(states)
    num_of_variables = 1
    # Allocate our to lists which will be the output:
    x = [] # The independent and dependent variables
    eta_list = [] # The function with the tangents        
    c = [] # The list of the coefficients
    # Allocate the polynomials which we will return as well
    polynomial_list = []
    # Define the time tangent
    xi = symbols('xi', cls=Function)
    # Add it to the tangent list
    eta_list.append(xi(t))    
    #----------------------------------------------------------------------------------
    # STEP 1: A LIST OF ALL INDEPENDENT AND DEPENDENT VARIABLES
    #----------------------------------------------------------------------------------
    substitution_list = [t]
    for state in states:
        substitution_list.append(state)
    #----------------------------------------------------------------------------------
    # STEP 2: GENERATE TEMPORARY VARIABLES    
    #----------------------------------------------------------------------------------
    # Add variables to the list
    for index in range(num_of_states+num_of_variables):
        # Allocate variable
        exec("x_%d = Symbol(\'x_%d\') "%(index,index))  
        #parse_expr("x_" + str(index) + " = Symbol(\'x_" + str(index) + "\')")
        # Add variable to the list    
        #parse_expr("x.append(x_" + str(index) + ")")
        exec("x.append(x_%d)"%(index)) 
    #----------------------------------------------------------------------------------
    # STEP 3: GENERATE POLYNOMIALS IN THE TEMPORARY VARIABLES
    #----------------------------------------------------------------------------------
    # Generate all monomials for our polynomials
    M = list(itermonomials(x, degree_polynomial))
    # Sort the list
    M = sorted(M, key=monomial_key('lex', x))
    # Calculate the number of terms in each of the polynomials
    num_of_terms = monomial_count(num_of_states+num_of_variables,degree_polynomial)
    #----------------------------------------------------------------------------------
    # STEP 4: DEFINE THE TANGENTIAL ANSÄTZE IN TERMS OF THE TEMPORARY VARIABLES
    #----------------------------------------------------------------------------------
    # Allocate our common polynomial
    P_0 = symbols('P_0', cls=Function)
    # Initialise it to zero
    P_0 = 0
    # Loop over all terms and add them to the common polynomial
    for index in range(num_of_terms):   
        # Allocate a coefficient
        exec("c_%d = symbols(\'c_%d\') "%(index,index)) 
        # Add this coefficient to the set of unknowns
        exec("c.append(c_%d)"%(index))
        # Add this coefficent to our polynomial at hand
        #exec("P_0+=c_%d*M[index]" % (index))
        P_0+=c[-1]*M[index]
    # Add our polynomial to our temporary list of polynomials
    polynomial_list.append(P_0)
    # Loop over the variables and create our tangent eta for each variable
    # and add it to the eta_list. Lastly, reset it to zero and start over 
    # again. 
    # Loop over our tangents
    for tangent_index in range(1,num_of_states+1):    
        # Allocate memory for our tangent
        exec("f_%d = symbols(\'f_%d\', cls=Function) "%(tangent_index,tangent_index)) 
        # Allocate memory for our function
        exec("f_%d = 0"%(tangent_index))
        # Loop over our two polynomials which define our ansatz
        exec("P_%d = symbols(\'P_%d\', cls=Function) "%(tangent_index,tangent_index)) 
        # Allocate memory for our newly created polynomial
        exec("P_%d=0"%(tangent_index))
        # Loop over the monomials and add them to the polynomial 
        for index in range(num_of_terms):   
            # Define a temporary index with the correct ordering
            temp_index = index + tangent_index*num_of_terms
            # Allocate a coefficient
            exec("c_%d = symbols(\'c_%d\') "%(temp_index,temp_index)) 
            # Add this coefficient to the set of unknowns
            exec("c.append(c_%d)"%(temp_index))
            # Add this coefficent to our polynomial at hand
            exec("P_%d+=c_%d*M[%d]" % (tangent_index,temp_index,index))
        # Add our polynomial to our temporary list of polynomials
        exec("polynomial_list.append(P_%d)"%(tangent_index))
        # Add the polynomial to our function
        exec("f_%d = P_%d*exp(P_0)"%(tangent_index,tangent_index))
        # Lastly, save our function
        exec("eta_list.append(f_%d)"% (tangent_index))
    #----------------------------------------------------------------------------------
    # STEP 5: TRANSLATE BACK TO THE ORIGINAL VARIABLES
    #----------------------------------------------------------------------------------        
    # Loop over all independent and dependent variables
    for variable_index in range(len(x)):
        # Loop over the monomials and conduct the substitutions
        for index in range(num_of_terms):
            # Substitute variable    
            M[index] = M[index].subs(x[variable_index],substitution_list[variable_index])
        # Loop over all tangents and do the substitutions
        for tangent_index in range(len(eta_list)):
            # Substitute variable
            eta_list[tangent_index] = eta_list[tangent_index].subs(x[variable_index],substitution_list[variable_index])
        for polynomial_index in range(len(polynomial_list)):
            # Same goes for polynomials
            polynomial_list[polynomial_index] = polynomial_list[polynomial_index].subs(x[variable_index],substitution_list[variable_index])
    # Return the output    
    return c, eta_list, M, polynomial_list
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# FUNCTION 4: "lin_sym_conds_fibre_preserving"
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# This function plugs in the ansatze for the tangents into the linearised symmetry
# conditions given an independent variable t, the states and the reaction terms.
def plug_in_ansatze_into_lin_sym(t,states,reaction_terms,tangent_list):
    # Temporary variables for the dependent variables so that we can 
    # differentiate the reaction terms with respect to each state
    x = symbols('x') 
    # Ok, so at this point we have our tangent lists, now we can start assembling our linearised symmetry conditions
    # Allocate memory for our linearised symmetry conditions
    lin_syms = []
    # Loop over the number of states and define the linearised symmetry condition
    for state_index in range(1,len(states)+1):
        # Define the LHS
        LHS = Derivative(tangent_list[state_index],t).doit() - omega_list[state_index-1]*Derivative(tangent_list[0],t).doit()
        # Allocate memory for the RHS
        RHS = 0
        # Add all partial derivatives of the current reaction term by looping over all states and times
        for RHS_index in range(1,len(states)+1):
            # Extract temporary tangent
            tangent_temp = tangent_list[RHS_index]
            # Partial derivative wrt to the time
            RHS += (Derivative(omega_list[state_index-1].subs(states[RHS_index-1],x),x).subs(x,states[RHS_index-1]).doit()*tangent_temp)
        # After this is done we loop over the states one more time and substitute all time derivatives of the
        # states with the reaction terms
        eq_temp = LHS - RHS
        for state_index in range(len(omega_list)):
            # Also, we substitute the reaction terms in the LHS
            eq_temp = eq_temp.subs(Derivative(states[state_index],t),omega_list[state_index])
        # Append the equation defined as LHS-RHS
        lin_syms.append((eq_temp,0))
    # Return the list with all the tangents
    return lin_syms
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# FUNCTION 5: "solve_algebraic_equations"
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# This function solves all the algebrac equations obtained after plugging in the ansatze
def solve_algebraic_equations(c,lin_sym_conds):
    # Make a copy of the coefficients
    c_copy = []
    for coeff_temp in c:
        c_copy.append(coeff_temp)
    # Allocate memory for all equations with constants in them
    eqs_coeffs = []
    # Allocate memory for all equations without constants in them
    eqs_coeffs_no_consts = []
    # Loop through all equations and sort them depending on whether or not they have constants in them
    for lin_sym_cond in lin_sym_conds:
        # Allocate a sum of all coefficients 
        coeff_sum = 0
        # Add all coefficients to this sum
        for coeff_temp in c:
            coeff_sum += lin_sym_cond[2].coeff(coeff_temp)
        # If we have coefficients we add them to the equations with coefficients
        if coeff_sum != 0:
            eqs_coeffs.append(lin_sym_cond)
        else:
            eqs_coeffs_no_consts.append(lin_sym_cond)
    # Make a copy of our equations
    eqs_coeffs_copy = [eqs_coeffs[index][2] for index in range(len(eqs_coeffs))]
    # Ok, we are ready! Let's loop through our equations, solve for each coefficients and substitute into
    # the remaining equations as well as coefficients.
    # Loop through all the equations
    #for eq_index in range(2):
    for eq_index in range(len(eqs_coeffs_copy)-1):
        # Extract the first equation
        eq_temp,denom = fraction(simplify(eqs_coeffs_copy[eq_index]))
        eq_temp = expand(eq_temp)
        # Coefficient we solve for
        coeff_we_solve_for = 0
        # Coeff index
        coeff_index = 0
        # Loop through all coefficients in c
        for coeff_temp in c:
            # Save the first coefficient occurring in our equation
            if eq_temp.coeff(coeff_temp)!=0:
                coeff_we_solve_for = coeff_temp
                break
        # Solve the current equation for the equation we solve for
        sol_temp = solve(eq_temp, coeff_we_solve_for)
        print("Equation:\t%s\t=\t0"%(eq_temp))
        print("Solution:\t%s\t=\t%s"%(str(coeff_we_solve_for),str(sol_temp[0])))
        # Loop through our coefficients, and substitue our solution
        for c_index in range(len(c_copy)):
            c_copy[c_index] = simplify(c_copy[c_index].subs(coeff_we_solve_for,sol_temp[0]))
        # Loop through the remaining equations and substitute this as well
        for eq_index_new in range(eq_index,len(eqs_coeffs_copy)):
            eqs_coeffs_copy[eq_index_new] = simplify(expand(eqs_coeffs_copy[eq_index_new].subs(coeff_we_solve_for,sol_temp[0])))
    # When the looping is done we return our new coefficients along with the sorted equations    
    return c_copy, eqs_coeffs, eqs_coeffs_no_consts

# Test output from automated function
Ansatze
\begin{align*}
\eta_1&=P_1\exp\left(P_0\right),\\
\eta_2&=P_2\exp\left(P_0\right).\\
\end{align*}
That is to say we have two unique polynomials, i.e. $P_1(t,u,v)$ and $P_2(t,u,v)$ of a given degree and one polynomial $P_0(t,u,v)$ which is common 

In [5]:
# Formulate the original linearised symmetry conditions and the list of tangents
tangent_list,lin_sym_original = lin_sym_conds_fibre_preserving(t,states,omega_list)    

In [6]:
# Create a nice output string for the original linearised symmetry conditions
tangent_str,lin_sym_original_str = latex_printing_of_lin_sym(t,states,tangent_list,lin_sym_original)
# Prompt to the user
print("The original linearised symmetry conditions are:")
print(lin_sym_original_str)

The original linearised symmetry conditions are:

\begin{align*}
a \left(u - 1\right) v \frac{\partial\eta_{1}}{\partial v} - \left(1 - v\right) \eta_{1} + \left(1 - v\right) u \frac{\partial\eta_{1}}{\partial u} - \left(1 - v\right) u \frac{d\xi}{dt} + \eta_{2} u + \frac{\partial\eta_1}{\partial t}&=0,\\
- a \left(u - 1\right) \eta_{2} + a \left(u - 1\right) v \frac{\partial\eta_{2}}{\partial v} - a \left(u - 1\right) v \frac{d\xi}{dt} - a \eta_{1} v + \left(1 - v\right) u \frac{\partial\eta_{2}}{\partial u} + \frac{\partial\eta_2}{\partial t}&=0,\\
\end{align*}




The original linearised symmetry conditions are:

\begin{align*}
a \left(u - 1\right) v \frac{\partial\eta_{1}}{\partial v} - \left(1 - v\right) \eta_{1} + \left(1 - v\right) u \frac{\partial\eta_{1}}{\partial u} - \left(1 - v\right) u \frac{d\xi}{dt} + \eta_{2} u + \frac{\partial\eta_1}{\partial t}&=0,\\
- a \left(u - 1\right) \eta_{2} + a \left(u - 1\right) v \frac{\partial\eta_{2}}{\partial v} - a \left(u - 1\right) v \frac{d\xi}{dt} - a \eta_{1} v + \left(1 - v\right) u \frac{\partial\eta_{2}}{\partial u} + \frac{\partial\eta_2}{\partial t}&=0,\\
\end{align*}

# Create tangential ansatze
Next, we want to test some polynomial ansatze to see if we can possibly simplify the linearised symmetry conditions. Since we already have tried the polynomials ansatze and these failed, we must modify these ansatze 

In [7]:
# Define the degree of the polynomial
degree_polynomial = 2
# Calculate the ansatze
c, eta_list, M, polynomial_list = create_tangent_ansatze(t,states,degree_polynomial)
# Print the degree of the polynomials and the unknown coefficients
print("We use ansatze of the type $$\\eta_i=P_{i}*\\exp\\left(P_{0}\\right)$$ for some index $i\\in\\left\\{1,\\ldots,n\\right\\}$ where $n$ is the number of states.")
print("Here, our two polynomials $P_{1}$ and $P_{2}$ are multivariate polynomials of a given degree. In this case, the degree of our polynomials are: $%d$."%(degree_polynomial))
print("Moreover, we have the following states or dependent variables:\n%s\nand one independent variable $t$."%(latex(Matrix(len(states),1,states),mode='equation').replace("\\begin{equation}","\\begin{equation}\n").replace("\\end{equation}","\n\\end{equation}")))
print("Now, given all variables (independent and dependent) as well as the degree of the polynomials, we obtain the following monomials:")
print(latex(Matrix(len(M),1,M),mode='equation').replace("\\begin{equation}","\\begin{equation}\nM=").replace("\\end{equation}",".\n\\end{equation}"))
print("Furthermore, we have the following unknown coefficients:")
print(latex(Matrix(len(c),1,c),mode='equation').replace("\\begin{equation}","\\begin{equation}\n\\mathbf{c}=").replace("\\end{equation}",".\n\\end{equation}"))
print("Now, given all of these unknowns and coefficient, this is the ansatze we get:")
print("\\begin{align*}")
for eta_index in range(len(eta_list)):
    temp_str = "\\eta_{%d}&=" + latex(eta_list[eta_index]) 
    if eta_index == len(eta_list)-1:
        temp_str += ".\\\\"
    else:
        temp_str += ",\\\\"        
    print(temp_str % (eta_index+1))
print("\\end{align*}")


We use ansatze of the type $$\eta_i=P_{i}*\exp\left(P_{0}\right)$$ for some index $i\in\left\{1,\ldots,n\right\}$ where $n$ is the number of states.
Here, our two polynomials $P_{1}$ and $P_{2}$ are multivariate polynomials of a given degree. In this case, the degree of our polynomials are: $2$.
Moreover, we have the following states or dependent variables:
\begin{equation}
\left[\begin{matrix}u{\left(t \right)}\\v{\left(t \right)}\end{matrix}\right]
\end{equation}
and one independent variable $t$.
Now, given all variables (independent and dependent) as well as the degree of the polynomials, we obtain the following monomials:
\begin{equation}
M=\left[\begin{matrix}1\\v{\left(t \right)}\\v^{2}{\left(t \right)}\\u{\left(t \right)}\\u{\left(t \right)} v{\left(t \right)}\\u^{2}{\left(t \right)}\\t\\t v{\left(t \right)}\\t u{\left(t \right)}\\t^{2}\end{matrix}\right].
\end{equation}
Furthermore, we have the following unknown coefficients:
\begin{equation}
\mathbf{c}=\left[\begin{matrix}c_{0

We use ansatze of the type $$\eta_i=P_{i}*\exp\left(P_{0}\right)$$ for some index $i\in\left\{1,\ldots,n\right\}$ where $n$ is the number of states.
Here, our two polynomials $P_{1}$ and $P_{2}$ are multivariate polynomials of a given degree. In this case, the degree of our polynomials are: $2$.
Moreover, we have the following states or dependent variables:
\begin{equation}
\left[\begin{matrix}u{\left(t \right)}\\v{\left(t \right)}\end{matrix}\right]
\end{equation}
and one independent variable $t$.
Now, given all variables (independent and dependent) as well as the degree of the polynomials, we obtain the following monomials:
\begin{equation}
M=\left[\begin{matrix}1\\v{\left(t \right)}\\v^{2}{\left(t \right)}\\u{\left(t \right)}\\u{\left(t \right)} v{\left(t \right)}\\u^{2}{\left(t \right)}\\t\\t v{\left(t \right)}\\t u{\left(t \right)}\\t^{2}\end{matrix}\right].
\end{equation}
Furthermore, we have the following unknown coefficients:
\begin{equation}
\mathbf{c}=\left[\begin{matrix}c_{0}\\c_{1}\\c_{2}\\c_{3}\\c_{4}\\c_{5}\\c_{6}\\c_{7}\\c_{8}\\c_{9}\\c_{10}\\c_{11}\\c_{12}\\c_{13}\\c_{14}\\c_{15}\\c_{16}\\c_{17}\\c_{18}\\c_{19}\\c_{20}\\c_{21}\\c_{22}\\c_{23}\\c_{24}\\c_{25}\\c_{26}\\c_{27}\\c_{28}\\c_{29}\end{matrix}\right].
\end{equation}
Now, given all of these unknowns and coefficient, this is the ansatze we get:
\begin{align*}
\eta_{1}&=\xi{\left(t \right)},\\
\eta_{2}&=\left(c_{10} + c_{11} v{\left(t \right)} + c_{12} v^{2}{\left(t \right)} + c_{13} u{\left(t \right)} + c_{14} u{\left(t \right)} v{\left(t \right)} + c_{15} u^{2}{\left(t \right)} + c_{16} t + c_{17} t v{\left(t \right)} + c_{18} t u{\left(t \right)} + c_{19} t^{2}\right) e^{c_{0} + c_{1} v{\left(t \right)} + c_{2} v^{2}{\left(t \right)} + c_{3} u{\left(t \right)} + c_{4} u{\left(t \right)} v{\left(t \right)} + c_{5} u^{2}{\left(t \right)} + c_{6} t + c_{7} t v{\left(t \right)} + c_{8} t u{\left(t \right)} + c_{9} t^{2}},\\
\eta_{3}&=\left(c_{20} + c_{21} v{\left(t \right)} + c_{22} v^{2}{\left(t \right)} + c_{23} u{\left(t \right)} + c_{24} u{\left(t \right)} v{\left(t \right)} + c_{25} u^{2}{\left(t \right)} + c_{26} t + c_{27} t v{\left(t \right)} + c_{28} t u{\left(t \right)} + c_{29} t^{2}\right) e^{c_{0} + c_{1} v{\left(t \right)} + c_{2} v^{2}{\left(t \right)} + c_{3} u{\left(t \right)} + c_{4} u{\left(t \right)} v{\left(t \right)} + c_{5} u^{2}{\left(t \right)} + c_{6} t + c_{7} t v{\left(t \right)} + c_{8} t u{\left(t \right)} + c_{9} t^{2}}.\\
\end{align*}

# Substituting these ansatze into the linearised symmetry conditions
Let's try to substitute these ansatze into the linearised symmetry conditions.


In [8]:
# Plug in our created ansatze into the linearised symmetry conditions
lin_syms = plug_in_ansatze_into_lin_sym(t,states,omega_list,eta_list)
# Tidy up these conditions by dividing by the common polynomial P_0
lin_syms_tidy = [expand(lin_sym[0]/exp(polynomial_list[0])) for lin_sym in lin_syms]
# Print our results
print("The linearised symmetry conditions when plugging in our tangential ansatze:")
print("\\begin{align*}")
for lin_sym in lin_syms_tidy:
    print("%s&=0\\\\"%(latex(lin_sym)))
print("\\end{align*}")
print("What happens when the monomial is 1?")
print(latex(lin_syms_tidy[0].coeff(M[0])))
print("Only extracting the exponential bit?")
print(latex(lin_syms_tidy[lin_sym_index].coeff(expand(1/exp(polynomial_list[0])))*expand(1/exp(polynomial_list[0]))))

The linearised symmetry conditions when plugging in our tangential ansatze:
\begin{align*}
a c_{1} c_{10} u{\left(t \right)} v{\left(t \right)} - a c_{1} c_{10} v{\left(t \right)} + a c_{1} c_{11} u{\left(t \right)} v^{2}{\left(t \right)} - a c_{1} c_{11} v^{2}{\left(t \right)} + a c_{1} c_{12} u{\left(t \right)} v^{3}{\left(t \right)} - a c_{1} c_{12} v^{3}{\left(t \right)} + a c_{1} c_{13} u^{2}{\left(t \right)} v{\left(t \right)} - a c_{1} c_{13} u{\left(t \right)} v{\left(t \right)} + a c_{1} c_{14} u^{2}{\left(t \right)} v^{2}{\left(t \right)} - a c_{1} c_{14} u{\left(t \right)} v^{2}{\left(t \right)} + a c_{1} c_{15} u^{3}{\left(t \right)} v{\left(t \right)} - a c_{1} c_{15} u^{2}{\left(t \right)} v{\left(t \right)} + a c_{1} c_{16} t u{\left(t \right)} v{\left(t \right)} - a c_{1} c_{16} t v{\left(t \right)} + a c_{1} c_{17} t u{\left(t \right)} v^{2}{\left(t \right)} - a c_{1} c_{17} t v^{2}{\left(t \right)} + a c_{1} c_{18} t u^{2}{\left(t \right)} v{\left(t \right)} - a c_{1}

a c_{1} c_{20} u{\left(t \right)} v{\left(t \right)} - a c_{1} c_{20} v{\left(t \right)} + a c_{1} c_{21} u{\left(t \right)} v^{2}{\left(t \right)} - a c_{1} c_{21} v^{2}{\left(t \right)} + a c_{1} c_{22} u{\left(t \right)} v^{3}{\left(t \right)} - a c_{1} c_{22} v^{3}{\left(t \right)} + a c_{1} c_{23} u^{2}{\left(t \right)} v{\left(t \right)} - a c_{1} c_{23} u{\left(t \right)} v{\left(t \right)} + a c_{1} c_{24} u^{2}{\left(t \right)} v^{2}{\left(t \right)} - a c_{1} c_{24} u{\left(t \right)} v^{2}{\left(t \right)} + a c_{1} c_{25} u^{3}{\left(t \right)} v{\left(t \right)} - a c_{1} c_{25} u^{2}{\left(t \right)} v{\left(t \right)} + a c_{1} c_{26} t u{\left(t \right)} v{\left(t \right)} - a c_{1} c_{26} t v{\left(t \right)} + a c_{1} c_{27} t u{\left(t \right)} v^{2}{\left(t \right)} - a c_{1} c_{27} t v^{2}{\left(t \right)} + a c_{1} c_{28} t u^{2}{\left(t \right)} v{\left(t \right)} - a c_{1} c_{28} t u{\left(t \right)} v{\left(t \right)} + a c_{1} c_{29} t^{2} u{\left(t \right)} v

NameError: name 'lin_sym_index' is not defined

The linearised symmetry conditions when plugging in our tangential ansatze:
\begin{align*}
a c_{1} c_{10} u{\left(t \right)} v{\left(t \right)} - a c_{1} c_{10} v{\left(t \right)} + a c_{1} c_{11} u{\left(t \right)} v^{2}{\left(t \right)} - a c_{1} c_{11} v^{2}{\left(t \right)} + a c_{1} c_{12} u{\left(t \right)} v^{3}{\left(t \right)} - a c_{1} c_{12} v^{3}{\left(t \right)} + a c_{1} c_{13} u^{2}{\left(t \right)} v{\left(t \right)} - a c_{1} c_{13} u{\left(t \right)} v{\left(t \right)} + a c_{1} c_{14} u^{2}{\left(t \right)} v^{2}{\left(t \right)} - a c_{1} c_{14} u{\left(t \right)} v^{2}{\left(t \right)} + a c_{1} c_{15} u^{3}{\left(t \right)} v{\left(t \right)} - a c_{1} c_{15} u^{2}{\left(t \right)} v{\left(t \right)} + a c_{1} c_{16} t u{\left(t \right)} v{\left(t \right)} - a c_{1} c_{16} t v{\left(t \right)} + a c_{1} c_{17} t u{\left(t \right)} v^{2}{\left(t \right)} - a c_{1} c_{17} t v^{2}{\left(t \right)} + a c_{1} c_{18} t u^{2}{\left(t \right)} v{\left(t \right)} - a c_{1} c_{18} t u{\left(t \right)} v{\left(t \right)} + a c_{1} c_{19} t^{2} u{\left(t \right)} v{\left(t \right)} - a c_{1} c_{19} t^{2} v{\left(t \right)} + 2 a c_{10} c_{2} u{\left(t \right)} v^{2}{\left(t \right)} - 2 a c_{10} c_{2} v^{2}{\left(t \right)} + a c_{10} c_{4} u^{2}{\left(t \right)} v{\left(t \right)} - a c_{10} c_{4} u{\left(t \right)} v{\left(t \right)} + a c_{10} c_{7} t u{\left(t \right)} v{\left(t \right)} - a c_{10} c_{7} t v{\left(t \right)} + 2 a c_{11} c_{2} u{\left(t \right)} v^{3}{\left(t \right)} - 2 a c_{11} c_{2} v^{3}{\left(t \right)} + a c_{11} c_{4} u^{2}{\left(t \right)} v^{2}{\left(t \right)} - a c_{11} c_{4} u{\left(t \right)} v^{2}{\left(t \right)} + a c_{11} c_{7} t u{\left(t \right)} v^{2}{\left(t \right)} - a c_{11} c_{7} t v^{2}{\left(t \right)} + a c_{11} u{\left(t \right)} v{\left(t \right)} - a c_{11} v{\left(t \right)} + 2 a c_{12} c_{2} u{\left(t \right)} v^{4}{\left(t \right)} - 2 a c_{12} c_{2} v^{4}{\left(t \right)} + a c_{12} c_{4} u^{2}{\left(t \right)} v^{3}{\left(t \right)} - a c_{12} c_{4} u{\left(t \right)} v^{3}{\left(t \right)} + a c_{12} c_{7} t u{\left(t \right)} v^{3}{\left(t \right)} - a c_{12} c_{7} t v^{3}{\left(t \right)} + 2 a c_{12} u{\left(t \right)} v^{2}{\left(t \right)} - 2 a c_{12} v^{2}{\left(t \right)} + 2 a c_{13} c_{2} u^{2}{\left(t \right)} v^{2}{\left(t \right)} - 2 a c_{13} c_{2} u{\left(t \right)} v^{2}{\left(t \right)} + a c_{13} c_{4} u^{3}{\left(t \right)} v{\left(t \right)} - a c_{13} c_{4} u^{2}{\left(t \right)} v{\left(t \right)} + a c_{13} c_{7} t u^{2}{\left(t \right)} v{\left(t \right)} - a c_{13} c_{7} t u{\left(t \right)} v{\left(t \right)} + 2 a c_{14} c_{2} u^{2}{\left(t \right)} v^{3}{\left(t \right)} - 2 a c_{14} c_{2} u{\left(t \right)} v^{3}{\left(t \right)} + a c_{14} c_{4} u^{3}{\left(t \right)} v^{2}{\left(t \right)} - a c_{14} c_{4} u^{2}{\left(t \right)} v^{2}{\left(t \right)} + a c_{14} c_{7} t u^{2}{\left(t \right)} v^{2}{\left(t \right)} - a c_{14} c_{7} t u{\left(t \right)} v^{2}{\left(t \right)} + a c_{14} u^{2}{\left(t \right)} v{\left(t \right)} - a c_{14} u{\left(t \right)} v{\left(t \right)} + 2 a c_{15} c_{2} u^{3}{\left(t \right)} v^{2}{\left(t \right)} - 2 a c_{15} c_{2} u^{2}{\left(t \right)} v^{2}{\left(t \right)} + a c_{15} c_{4} u^{4}{\left(t \right)} v{\left(t \right)} - a c_{15} c_{4} u^{3}{\left(t \right)} v{\left(t \right)} + a c_{15} c_{7} t u^{3}{\left(t \right)} v{\left(t \right)} - a c_{15} c_{7} t u^{2}{\left(t \right)} v{\left(t \right)} + 2 a c_{16} c_{2} t u{\left(t \right)} v^{2}{\left(t \right)} - 2 a c_{16} c_{2} t v^{2}{\left(t \right)} + a c_{16} c_{4} t u^{2}{\left(t \right)} v{\left(t \right)} - a c_{16} c_{4} t u{\left(t \right)} v{\left(t \right)} + a c_{16} c_{7} t^{2} u{\left(t \right)} v{\left(t \right)} - a c_{16} c_{7} t^{2} v{\left(t \right)} + 2 a c_{17} c_{2} t u{\left(t \right)} v^{3}{\left(t \right)} - 2 a c_{17} c_{2} t v^{3}{\left(t \right)} + a c_{17} c_{4} t u^{2}{\left(t \right)} v^{2}{\left(t \right)} - a c_{17} c_{4} t u{\left(t \right)} v^{2}{\left(t \right)} + a c_{17} c_{7} t^{2} u{\left(t \right)} v^{2}{\left(t \right)} - a c_{17} c_{7} t^{2} v^{2}{\left(t \right)} + a c_{17} t u{\left(t \right)} v{\left(t \right)} - a c_{17} t v{\left(t \right)} + 2 a c_{18} c_{2} t u^{2}{\left(t \right)} v^{2}{\left(t \right)} - 2 a c_{18} c_{2} t u{\left(t \right)} v^{2}{\left(t \right)} + a c_{18} c_{4} t u^{3}{\left(t \right)} v{\left(t \right)} - a c_{18} c_{4} t u^{2}{\left(t \right)} v{\left(t \right)} + a c_{18} c_{7} t^{2} u^{2}{\left(t \right)} v{\left(t \right)} - a c_{18} c_{7} t^{2} u{\left(t \right)} v{\left(t \right)} + 2 a c_{19} c_{2} t^{2} u{\left(t \right)} v^{2}{\left(t \right)} - 2 a c_{19} c_{2} t^{2} v^{2}{\left(t \right)} + a c_{19} c_{4} t^{2} u^{2}{\left(t \right)} v{\left(t \right)} - a c_{19} c_{4} t^{2} u{\left(t \right)} v{\left(t \right)} + a c_{19} c_{7} t^{3} u{\left(t \right)} v{\left(t \right)} - a c_{19} c_{7} t^{3} v{\left(t \right)} - c_{10} c_{3} u{\left(t \right)} v{\left(t \right)} + c_{10} c_{3} u{\left(t \right)} - c_{10} c_{4} u{\left(t \right)} v^{2}{\left(t \right)} + c_{10} c_{4} u{\left(t \right)} v{\left(t \right)} - 2 c_{10} c_{5} u^{2}{\left(t \right)} v{\left(t \right)} + 2 c_{10} c_{5} u^{2}{\left(t \right)} + c_{10} c_{6} + c_{10} c_{7} v{\left(t \right)} - c_{10} c_{8} t u{\left(t \right)} v{\left(t \right)} + c_{10} c_{8} t u{\left(t \right)} + c_{10} c_{8} u{\left(t \right)} + 2 c_{10} c_{9} t + c_{10} v{\left(t \right)} - c_{10} - c_{11} c_{3} u{\left(t \right)} v^{2}{\left(t \right)} + c_{11} c_{3} u{\left(t \right)} v{\left(t \right)} - c_{11} c_{4} u{\left(t \right)} v^{3}{\left(t \right)} + c_{11} c_{4} u{\left(t \right)} v^{2}{\left(t \right)} - 2 c_{11} c_{5} u^{2}{\left(t \right)} v^{2}{\left(t \right)} + 2 c_{11} c_{5} u^{2}{\left(t \right)} v{\left(t \right)} + c_{11} c_{6} v{\left(t \right)} + c_{11} c_{7} v^{2}{\left(t \right)} - c_{11} c_{8} t u{\left(t \right)} v^{2}{\left(t \right)} + c_{11} c_{8} t u{\left(t \right)} v{\left(t \right)} + c_{11} c_{8} u{\left(t \right)} v{\left(t \right)} + 2 c_{11} c_{9} t v{\left(t \right)} + c_{11} v^{2}{\left(t \right)} - c_{11} v{\left(t \right)} - c_{12} c_{3} u{\left(t \right)} v^{3}{\left(t \right)} + c_{12} c_{3} u{\left(t \right)} v^{2}{\left(t \right)} - c_{12} c_{4} u{\left(t \right)} v^{4}{\left(t \right)} + c_{12} c_{4} u{\left(t \right)} v^{3}{\left(t \right)} - 2 c_{12} c_{5} u^{2}{\left(t \right)} v^{3}{\left(t \right)} + 2 c_{12} c_{5} u^{2}{\left(t \right)} v^{2}{\left(t \right)} + c_{12} c_{6} v^{2}{\left(t \right)} + c_{12} c_{7} v^{3}{\left(t \right)} - c_{12} c_{8} t u{\left(t \right)} v^{3}{\left(t \right)} + c_{12} c_{8} t u{\left(t \right)} v^{2}{\left(t \right)} + c_{12} c_{8} u{\left(t \right)} v^{2}{\left(t \right)} + 2 c_{12} c_{9} t v^{2}{\left(t \right)} + c_{12} v^{3}{\left(t \right)} - c_{12} v^{2}{\left(t \right)} - c_{13} c_{3} u^{2}{\left(t \right)} v{\left(t \right)} + c_{13} c_{3} u^{2}{\left(t \right)} - c_{13} c_{4} u^{2}{\left(t \right)} v^{2}{\left(t \right)} + c_{13} c_{4} u^{2}{\left(t \right)} v{\left(t \right)} - 2 c_{13} c_{5} u^{3}{\left(t \right)} v{\left(t \right)} + 2 c_{13} c_{5} u^{3}{\left(t \right)} + c_{13} c_{6} u{\left(t \right)} + c_{13} c_{7} u{\left(t \right)} v{\left(t \right)} - c_{13} c_{8} t u^{2}{\left(t \right)} v{\left(t \right)} + c_{13} c_{8} t u^{2}{\left(t \right)} + c_{13} c_{8} u^{2}{\left(t \right)} + 2 c_{13} c_{9} t u{\left(t \right)} - c_{14} c_{3} u^{2}{\left(t \right)} v^{2}{\left(t \right)} + c_{14} c_{3} u^{2}{\left(t \right)} v{\left(t \right)} - c_{14} c_{4} u^{2}{\left(t \right)} v^{3}{\left(t \right)} + c_{14} c_{4} u^{2}{\left(t \right)} v^{2}{\left(t \right)} - 2 c_{14} c_{5} u^{3}{\left(t \right)} v^{2}{\left(t \right)} + 2 c_{14} c_{5} u^{3}{\left(t \right)} v{\left(t \right)} + c_{14} c_{6} u{\left(t \right)} v{\left(t \right)} + c_{14} c_{7} u{\left(t \right)} v^{2}{\left(t \right)} - c_{14} c_{8} t u^{2}{\left(t \right)} v^{2}{\left(t \right)} + c_{14} c_{8} t u^{2}{\left(t \right)} v{\left(t \right)} + c_{14} c_{8} u^{2}{\left(t \right)} v{\left(t \right)} + 2 c_{14} c_{9} t u{\left(t \right)} v{\left(t \right)} - c_{15} c_{3} u^{3}{\left(t \right)} v{\left(t \right)} + c_{15} c_{3} u^{3}{\left(t \right)} - c_{15} c_{4} u^{3}{\left(t \right)} v^{2}{\left(t \right)} + c_{15} c_{4} u^{3}{\left(t \right)} v{\left(t \right)} - 2 c_{15} c_{5} u^{4}{\left(t \right)} v{\left(t \right)} + 2 c_{15} c_{5} u^{4}{\left(t \right)} + c_{15} c_{6} u^{2}{\left(t \right)} + c_{15} c_{7} u^{2}{\left(t \right)} v{\left(t \right)} - c_{15} c_{8} t u^{3}{\left(t \right)} v{\left(t \right)} + c_{15} c_{8} t u^{3}{\left(t \right)} + c_{15} c_{8} u^{3}{\left(t \right)} + 2 c_{15} c_{9} t u^{2}{\left(t \right)} - c_{15} u^{2}{\left(t \right)} v{\left(t \right)} + c_{15} u^{2}{\left(t \right)} - c_{16} c_{3} t u{\left(t \right)} v{\left(t \right)} + c_{16} c_{3} t u{\left(t \right)} - c_{16} c_{4} t u{\left(t \right)} v^{2}{\left(t \right)} + c_{16} c_{4} t u{\left(t \right)} v{\left(t \right)} - 2 c_{16} c_{5} t u^{2}{\left(t \right)} v{\left(t \right)} + 2 c_{16} c_{5} t u^{2}{\left(t \right)} + c_{16} c_{6} t + c_{16} c_{7} t v{\left(t \right)} - c_{16} c_{8} t^{2} u{\left(t \right)} v{\left(t \right)} + c_{16} c_{8} t^{2} u{\left(t \right)} + c_{16} c_{8} t u{\left(t \right)} + 2 c_{16} c_{9} t^{2} + c_{16} t v{\left(t \right)} - c_{16} t + c_{16} - c_{17} c_{3} t u{\left(t \right)} v^{2}{\left(t \right)} + c_{17} c_{3} t u{\left(t \right)} v{\left(t \right)} - c_{17} c_{4} t u{\left(t \right)} v^{3}{\left(t \right)} + c_{17} c_{4} t u{\left(t \right)} v^{2}{\left(t \right)} - 2 c_{17} c_{5} t u^{2}{\left(t \right)} v^{2}{\left(t \right)} + 2 c_{17} c_{5} t u^{2}{\left(t \right)} v{\left(t \right)} + c_{17} c_{6} t v{\left(t \right)} + c_{17} c_{7} t v^{2}{\left(t \right)} - c_{17} c_{8} t^{2} u{\left(t \right)} v^{2}{\left(t \right)} + c_{17} c_{8} t^{2} u{\left(t \right)} v{\left(t \right)} + c_{17} c_{8} t u{\left(t \right)} v{\left(t \right)} + 2 c_{17} c_{9} t^{2} v{\left(t \right)} + c_{17} t v^{2}{\left(t \right)} - c_{17} t v{\left(t \right)} + c_{17} v{\left(t \right)} - c_{18} c_{3} t u^{2}{\left(t \right)} v{\left(t \right)} + c_{18} c_{3} t u^{2}{\left(t \right)} - c_{18} c_{4} t u^{2}{\left(t \right)} v^{2}{\left(t \right)} + c_{18} c_{4} t u^{2}{\left(t \right)} v{\left(t \right)} - 2 c_{18} c_{5} t u^{3}{\left(t \right)} v{\left(t \right)} + 2 c_{18} c_{5} t u^{3}{\left(t \right)} + c_{18} c_{6} t u{\left(t \right)} + c_{18} c_{7} t u{\left(t \right)} v{\left(t \right)} - c_{18} c_{8} t^{2} u^{2}{\left(t \right)} v{\left(t \right)} + c_{18} c_{8} t^{2} u^{2}{\left(t \right)} + c_{18} c_{8} t u^{2}{\left(t \right)} + 2 c_{18} c_{9} t^{2} u{\left(t \right)} + c_{18} u{\left(t \right)} - c_{19} c_{3} t^{2} u{\left(t \right)} v{\left(t \right)} + c_{19} c_{3} t^{2} u{\left(t \right)} - c_{19} c_{4} t^{2} u{\left(t \right)} v^{2}{\left(t \right)} + c_{19} c_{4} t^{2} u{\left(t \right)} v{\left(t \right)} - 2 c_{19} c_{5} t^{2} u^{2}{\left(t \right)} v{\left(t \right)} + 2 c_{19} c_{5} t^{2} u^{2}{\left(t \right)} + c_{19} c_{6} t^{2} + c_{19} c_{7} t^{2} v{\left(t \right)} - c_{19} c_{8} t^{3} u{\left(t \right)} v{\left(t \right)} + c_{19} c_{8} t^{3} u{\left(t \right)} + c_{19} c_{8} t^{2} u{\left(t \right)} + 2 c_{19} c_{9} t^{3} + c_{19} t^{2} v{\left(t \right)} - c_{19} t^{2} + 2 c_{19} t + c_{20} u{\left(t \right)} + c_{21} u{\left(t \right)} v{\left(t \right)} + c_{22} u{\left(t \right)} v^{2}{\left(t \right)} + c_{23} u^{2}{\left(t \right)} + c_{24} u^{2}{\left(t \right)} v{\left(t \right)} + c_{25} u^{3}{\left(t \right)} + c_{26} t u{\left(t \right)} + c_{27} t u{\left(t \right)} v{\left(t \right)} + c_{28} t u^{2}{\left(t \right)} + c_{29} t^{2} u{\left(t \right)} + u{\left(t \right)} v{\left(t \right)} e^{- c_{0}} e^{- c_{1} v{\left(t \right)}} e^{- c_{2} v^{2}{\left(t \right)}} e^{- c_{3} u{\left(t \right)}} e^{- c_{5} u^{2}{\left(t \right)}} e^{- c_{6} t} e^{- c_{9} t^{2}} e^{- c_{4} u{\left(t \right)} v{\left(t \right)}} e^{- c_{7} t v{\left(t \right)}} e^{- c_{8} t u{\left(t \right)}} \frac{d}{d t} \xi{\left(t \right)} - u{\left(t \right)} e^{- c_{0}} e^{- c_{1} v{\left(t \right)}} e^{- c_{2} v^{2}{\left(t \right)}} e^{- c_{3} u{\left(t \right)}} e^{- c_{5} u^{2}{\left(t \right)}} e^{- c_{6} t} e^{- c_{9} t^{2}} e^{- c_{4} u{\left(t \right)} v{\left(t \right)}} e^{- c_{7} t v{\left(t \right)}} e^{- c_{8} t u{\left(t \right)}} \frac{d}{d t} \xi{\left(t \right)}&=0,\\
a c_{1} c_{20} u{\left(t \right)} v{\left(t \right)} - a c_{1} c_{20} v{\left(t \right)} + a c_{1} c_{21} u{\left(t \right)} v^{2}{\left(t \right)} - a c_{1} c_{21} v^{2}{\left(t \right)} + a c_{1} c_{22} u{\left(t \right)} v^{3}{\left(t \right)} - a c_{1} c_{22} v^{3}{\left(t \right)} + a c_{1} c_{23} u^{2}{\left(t \right)} v{\left(t \right)} - a c_{1} c_{23} u{\left(t \right)} v{\left(t \right)} + a c_{1} c_{24} u^{2}{\left(t \right)} v^{2}{\left(t \right)} - a c_{1} c_{24} u{\left(t \right)} v^{2}{\left(t \right)} + a c_{1} c_{25} u^{3}{\left(t \right)} v{\left(t \right)} - a c_{1} c_{25} u^{2}{\left(t \right)} v{\left(t \right)} + a c_{1} c_{26} t u{\left(t \right)} v{\left(t \right)} - a c_{1} c_{26} t v{\left(t \right)} + a c_{1} c_{27} t u{\left(t \right)} v^{2}{\left(t \right)} - a c_{1} c_{27} t v^{2}{\left(t \right)} + a c_{1} c_{28} t u^{2}{\left(t \right)} v{\left(t \right)} - a c_{1} c_{28} t u{\left(t \right)} v{\left(t \right)} + a c_{1} c_{29} t^{2} u{\left(t \right)} v{\left(t \right)} - a c_{1} c_{29} t^{2} v{\left(t \right)} - a c_{10} v{\left(t \right)} - a c_{11} v^{2}{\left(t \right)} - a c_{12} v^{3}{\left(t \right)} - a c_{13} u{\left(t \right)} v{\left(t \right)} - a c_{14} u{\left(t \right)} v^{2}{\left(t \right)} - a c_{15} u^{2}{\left(t \right)} v{\left(t \right)} - a c_{16} t v{\left(t \right)} - a c_{17} t v^{2}{\left(t \right)} - a c_{18} t u{\left(t \right)} v{\left(t \right)} - a c_{19} t^{2} v{\left(t \right)} + 2 a c_{2} c_{20} u{\left(t \right)} v^{2}{\left(t \right)} - 2 a c_{2} c_{20} v^{2}{\left(t \right)} + 2 a c_{2} c_{21} u{\left(t \right)} v^{3}{\left(t \right)} - 2 a c_{2} c_{21} v^{3}{\left(t \right)} + 2 a c_{2} c_{22} u{\left(t \right)} v^{4}{\left(t \right)} - 2 a c_{2} c_{22} v^{4}{\left(t \right)} + 2 a c_{2} c_{23} u^{2}{\left(t \right)} v^{2}{\left(t \right)} - 2 a c_{2} c_{23} u{\left(t \right)} v^{2}{\left(t \right)} + 2 a c_{2} c_{24} u^{2}{\left(t \right)} v^{3}{\left(t \right)} - 2 a c_{2} c_{24} u{\left(t \right)} v^{3}{\left(t \right)} + 2 a c_{2} c_{25} u^{3}{\left(t \right)} v^{2}{\left(t \right)} - 2 a c_{2} c_{25} u^{2}{\left(t \right)} v^{2}{\left(t \right)} + 2 a c_{2} c_{26} t u{\left(t \right)} v^{2}{\left(t \right)} - 2 a c_{2} c_{26} t v^{2}{\left(t \right)} + 2 a c_{2} c_{27} t u{\left(t \right)} v^{3}{\left(t \right)} - 2 a c_{2} c_{27} t v^{3}{\left(t \right)} + 2 a c_{2} c_{28} t u^{2}{\left(t \right)} v^{2}{\left(t \right)} - 2 a c_{2} c_{28} t u{\left(t \right)} v^{2}{\left(t \right)} + 2 a c_{2} c_{29} t^{2} u{\left(t \right)} v^{2}{\left(t \right)} - 2 a c_{2} c_{29} t^{2} v^{2}{\left(t \right)} + a c_{20} c_{4} u^{2}{\left(t \right)} v{\left(t \right)} - a c_{20} c_{4} u{\left(t \right)} v{\left(t \right)} + a c_{20} c_{7} t u{\left(t \right)} v{\left(t \right)} - a c_{20} c_{7} t v{\left(t \right)} - a c_{20} u{\left(t \right)} + a c_{20} + a c_{21} c_{4} u^{2}{\left(t \right)} v^{2}{\left(t \right)} - a c_{21} c_{4} u{\left(t \right)} v^{2}{\left(t \right)} + a c_{21} c_{7} t u{\left(t \right)} v^{2}{\left(t \right)} - a c_{21} c_{7} t v^{2}{\left(t \right)} + a c_{22} c_{4} u^{2}{\left(t \right)} v^{3}{\left(t \right)} - a c_{22} c_{4} u{\left(t \right)} v^{3}{\left(t \right)} + a c_{22} c_{7} t u{\left(t \right)} v^{3}{\left(t \right)} - a c_{22} c_{7} t v^{3}{\left(t \right)} + a c_{22} u{\left(t \right)} v^{2}{\left(t \right)} - a c_{22} v^{2}{\left(t \right)} + a c_{23} c_{4} u^{3}{\left(t \right)} v{\left(t \right)} - a c_{23} c_{4} u^{2}{\left(t \right)} v{\left(t \right)} + a c_{23} c_{7} t u^{2}{\left(t \right)} v{\left(t \right)} - a c_{23} c_{7} t u{\left(t \right)} v{\left(t \right)} - a c_{23} u^{2}{\left(t \right)} + a c_{23} u{\left(t \right)} + a c_{24} c_{4} u^{3}{\left(t \right)} v^{2}{\left(t \right)} - a c_{24} c_{4} u^{2}{\left(t \right)} v^{2}{\left(t \right)} + a c_{24} c_{7} t u^{2}{\left(t \right)} v^{2}{\left(t \right)} - a c_{24} c_{7} t u{\left(t \right)} v^{2}{\left(t \right)} + a c_{25} c_{4} u^{4}{\left(t \right)} v{\left(t \right)} - a c_{25} c_{4} u^{3}{\left(t \right)} v{\left(t \right)} + a c_{25} c_{7} t u^{3}{\left(t \right)} v{\left(t \right)} - a c_{25} c_{7} t u^{2}{\left(t \right)} v{\left(t \right)} - a c_{25} u^{3}{\left(t \right)} + a c_{25} u^{2}{\left(t \right)} + a c_{26} c_{4} t u^{2}{\left(t \right)} v{\left(t \right)} - a c_{26} c_{4} t u{\left(t \right)} v{\left(t \right)} + a c_{26} c_{7} t^{2} u{\left(t \right)} v{\left(t \right)} - a c_{26} c_{7} t^{2} v{\left(t \right)} - a c_{26} t u{\left(t \right)} + a c_{26} t + a c_{27} c_{4} t u^{2}{\left(t \right)} v^{2}{\left(t \right)} - a c_{27} c_{4} t u{\left(t \right)} v^{2}{\left(t \right)} + a c_{27} c_{7} t^{2} u{\left(t \right)} v^{2}{\left(t \right)} - a c_{27} c_{7} t^{2} v^{2}{\left(t \right)} + a c_{28} c_{4} t u^{3}{\left(t \right)} v{\left(t \right)} - a c_{28} c_{4} t u^{2}{\left(t \right)} v{\left(t \right)} + a c_{28} c_{7} t^{2} u^{2}{\left(t \right)} v{\left(t \right)} - a c_{28} c_{7} t^{2} u{\left(t \right)} v{\left(t \right)} - a c_{28} t u^{2}{\left(t \right)} + a c_{28} t u{\left(t \right)} + a c_{29} c_{4} t^{2} u^{2}{\left(t \right)} v{\left(t \right)} - a c_{29} c_{4} t^{2} u{\left(t \right)} v{\left(t \right)} + a c_{29} c_{7} t^{3} u{\left(t \right)} v{\left(t \right)} - a c_{29} c_{7} t^{3} v{\left(t \right)} - a c_{29} t^{2} u{\left(t \right)} + a c_{29} t^{2} - a u{\left(t \right)} v{\left(t \right)} e^{- c_{0}} e^{- c_{1} v{\left(t \right)}} e^{- c_{2} v^{2}{\left(t \right)}} e^{- c_{3} u{\left(t \right)}} e^{- c_{5} u^{2}{\left(t \right)}} e^{- c_{6} t} e^{- c_{9} t^{2}} e^{- c_{4} u{\left(t \right)} v{\left(t \right)}} e^{- c_{7} t v{\left(t \right)}} e^{- c_{8} t u{\left(t \right)}} \frac{d}{d t} \xi{\left(t \right)} + a v{\left(t \right)} e^{- c_{0}} e^{- c_{1} v{\left(t \right)}} e^{- c_{2} v^{2}{\left(t \right)}} e^{- c_{3} u{\left(t \right)}} e^{- c_{5} u^{2}{\left(t \right)}} e^{- c_{6} t} e^{- c_{9} t^{2}} e^{- c_{4} u{\left(t \right)} v{\left(t \right)}} e^{- c_{7} t v{\left(t \right)}} e^{- c_{8} t u{\left(t \right)}} \frac{d}{d t} \xi{\left(t \right)} - c_{20} c_{3} u{\left(t \right)} v{\left(t \right)} + c_{20} c_{3} u{\left(t \right)} - c_{20} c_{4} u{\left(t \right)} v^{2}{\left(t \right)} + c_{20} c_{4} u{\left(t \right)} v{\left(t \right)} - 2 c_{20} c_{5} u^{2}{\left(t \right)} v{\left(t \right)} + 2 c_{20} c_{5} u^{2}{\left(t \right)} + c_{20} c_{6} + c_{20} c_{7} v{\left(t \right)} - c_{20} c_{8} t u{\left(t \right)} v{\left(t \right)} + c_{20} c_{8} t u{\left(t \right)} + c_{20} c_{8} u{\left(t \right)} + 2 c_{20} c_{9} t - c_{21} c_{3} u{\left(t \right)} v^{2}{\left(t \right)} + c_{21} c_{3} u{\left(t \right)} v{\left(t \right)} - c_{21} c_{4} u{\left(t \right)} v^{3}{\left(t \right)} + c_{21} c_{4} u{\left(t \right)} v^{2}{\left(t \right)} - 2 c_{21} c_{5} u^{2}{\left(t \right)} v^{2}{\left(t \right)} + 2 c_{21} c_{5} u^{2}{\left(t \right)} v{\left(t \right)} + c_{21} c_{6} v{\left(t \right)} + c_{21} c_{7} v^{2}{\left(t \right)} - c_{21} c_{8} t u{\left(t \right)} v^{2}{\left(t \right)} + c_{21} c_{8} t u{\left(t \right)} v{\left(t \right)} + c_{21} c_{8} u{\left(t \right)} v{\left(t \right)} + 2 c_{21} c_{9} t v{\left(t \right)} - c_{22} c_{3} u{\left(t \right)} v^{3}{\left(t \right)} + c_{22} c_{3} u{\left(t \right)} v^{2}{\left(t \right)} - c_{22} c_{4} u{\left(t \right)} v^{4}{\left(t \right)} + c_{22} c_{4} u{\left(t \right)} v^{3}{\left(t \right)} - 2 c_{22} c_{5} u^{2}{\left(t \right)} v^{3}{\left(t \right)} + 2 c_{22} c_{5} u^{2}{\left(t \right)} v^{2}{\left(t \right)} + c_{22} c_{6} v^{2}{\left(t \right)} + c_{22} c_{7} v^{3}{\left(t \right)} - c_{22} c_{8} t u{\left(t \right)} v^{3}{\left(t \right)} + c_{22} c_{8} t u{\left(t \right)} v^{2}{\left(t \right)} + c_{22} c_{8} u{\left(t \right)} v^{2}{\left(t \right)} + 2 c_{22} c_{9} t v^{2}{\left(t \right)} - c_{23} c_{3} u^{2}{\left(t \right)} v{\left(t \right)} + c_{23} c_{3} u^{2}{\left(t \right)} - c_{23} c_{4} u^{2}{\left(t \right)} v^{2}{\left(t \right)} + c_{23} c_{4} u^{2}{\left(t \right)} v{\left(t \right)} - 2 c_{23} c_{5} u^{3}{\left(t \right)} v{\left(t \right)} + 2 c_{23} c_{5} u^{3}{\left(t \right)} + c_{23} c_{6} u{\left(t \right)} + c_{23} c_{7} u{\left(t \right)} v{\left(t \right)} - c_{23} c_{8} t u^{2}{\left(t \right)} v{\left(t \right)} + c_{23} c_{8} t u^{2}{\left(t \right)} + c_{23} c_{8} u^{2}{\left(t \right)} + 2 c_{23} c_{9} t u{\left(t \right)} - c_{23} u{\left(t \right)} v{\left(t \right)} + c_{23} u{\left(t \right)} - c_{24} c_{3} u^{2}{\left(t \right)} v^{2}{\left(t \right)} + c_{24} c_{3} u^{2}{\left(t \right)} v{\left(t \right)} - c_{24} c_{4} u^{2}{\left(t \right)} v^{3}{\left(t \right)} + c_{24} c_{4} u^{2}{\left(t \right)} v^{2}{\left(t \right)} - 2 c_{24} c_{5} u^{3}{\left(t \right)} v^{2}{\left(t \right)} + 2 c_{24} c_{5} u^{3}{\left(t \right)} v{\left(t \right)} + c_{24} c_{6} u{\left(t \right)} v{\left(t \right)} + c_{24} c_{7} u{\left(t \right)} v^{2}{\left(t \right)} - c_{24} c_{8} t u^{2}{\left(t \right)} v^{2}{\left(t \right)} + c_{24} c_{8} t u^{2}{\left(t \right)} v{\left(t \right)} + c_{24} c_{8} u^{2}{\left(t \right)} v{\left(t \right)} + 2 c_{24} c_{9} t u{\left(t \right)} v{\left(t \right)} - c_{24} u{\left(t \right)} v^{2}{\left(t \right)} + c_{24} u{\left(t \right)} v{\left(t \right)} - c_{25} c_{3} u^{3}{\left(t \right)} v{\left(t \right)} + c_{25} c_{3} u^{3}{\left(t \right)} - c_{25} c_{4} u^{3}{\left(t \right)} v^{2}{\left(t \right)} + c_{25} c_{4} u^{3}{\left(t \right)} v{\left(t \right)} - 2 c_{25} c_{5} u^{4}{\left(t \right)} v{\left(t \right)} + 2 c_{25} c_{5} u^{4}{\left(t \right)} + c_{25} c_{6} u^{2}{\left(t \right)} + c_{25} c_{7} u^{2}{\left(t \right)} v{\left(t \right)} - c_{25} c_{8} t u^{3}{\left(t \right)} v{\left(t \right)} + c_{25} c_{8} t u^{3}{\left(t \right)} + c_{25} c_{8} u^{3}{\left(t \right)} + 2 c_{25} c_{9} t u^{2}{\left(t \right)} - 2 c_{25} u^{2}{\left(t \right)} v{\left(t \right)} + 2 c_{25} u^{2}{\left(t \right)} - c_{26} c_{3} t u{\left(t \right)} v{\left(t \right)} + c_{26} c_{3} t u{\left(t \right)} - c_{26} c_{4} t u{\left(t \right)} v^{2}{\left(t \right)} + c_{26} c_{4} t u{\left(t \right)} v{\left(t \right)} - 2 c_{26} c_{5} t u^{2}{\left(t \right)} v{\left(t \right)} + 2 c_{26} c_{5} t u^{2}{\left(t \right)} + c_{26} c_{6} t + c_{26} c_{7} t v{\left(t \right)} - c_{26} c_{8} t^{2} u{\left(t \right)} v{\left(t \right)} + c_{26} c_{8} t^{2} u{\left(t \right)} + c_{26} c_{8} t u{\left(t \right)} + 2 c_{26} c_{9} t^{2} + c_{26} - c_{27} c_{3} t u{\left(t \right)} v^{2}{\left(t \right)} + c_{27} c_{3} t u{\left(t \right)} v{\left(t \right)} - c_{27} c_{4} t u{\left(t \right)} v^{3}{\left(t \right)} + c_{27} c_{4} t u{\left(t \right)} v^{2}{\left(t \right)} - 2 c_{27} c_{5} t u^{2}{\left(t \right)} v^{2}{\left(t \right)} + 2 c_{27} c_{5} t u^{2}{\left(t \right)} v{\left(t \right)} + c_{27} c_{6} t v{\left(t \right)} + c_{27} c_{7} t v^{2}{\left(t \right)} - c_{27} c_{8} t^{2} u{\left(t \right)} v^{2}{\left(t \right)} + c_{27} c_{8} t^{2} u{\left(t \right)} v{\left(t \right)} + c_{27} c_{8} t u{\left(t \right)} v{\left(t \right)} + 2 c_{27} c_{9} t^{2} v{\left(t \right)} + c_{27} v{\left(t \right)} - c_{28} c_{3} t u^{2}{\left(t \right)} v{\left(t \right)} + c_{28} c_{3} t u^{2}{\left(t \right)} - c_{28} c_{4} t u^{2}{\left(t \right)} v^{2}{\left(t \right)} + c_{28} c_{4} t u^{2}{\left(t \right)} v{\left(t \right)} - 2 c_{28} c_{5} t u^{3}{\left(t \right)} v{\left(t \right)} + 2 c_{28} c_{5} t u^{3}{\left(t \right)} + c_{28} c_{6} t u{\left(t \right)} + c_{28} c_{7} t u{\left(t \right)} v{\left(t \right)} - c_{28} c_{8} t^{2} u^{2}{\left(t \right)} v{\left(t \right)} + c_{28} c_{8} t^{2} u^{2}{\left(t \right)} + c_{28} c_{8} t u^{2}{\left(t \right)} + 2 c_{28} c_{9} t^{2} u{\left(t \right)} - c_{28} t u{\left(t \right)} v{\left(t \right)} + c_{28} t u{\left(t \right)} + c_{28} u{\left(t \right)} - c_{29} c_{3} t^{2} u{\left(t \right)} v{\left(t \right)} + c_{29} c_{3} t^{2} u{\left(t \right)} - c_{29} c_{4} t^{2} u{\left(t \right)} v^{2}{\left(t \right)} + c_{29} c_{4} t^{2} u{\left(t \right)} v{\left(t \right)} - 2 c_{29} c_{5} t^{2} u^{2}{\left(t \right)} v{\left(t \right)} + 2 c_{29} c_{5} t^{2} u^{2}{\left(t \right)} + c_{29} c_{6} t^{2} + c_{29} c_{7} t^{2} v{\left(t \right)} - c_{29} c_{8} t^{3} u{\left(t \right)} v{\left(t \right)} + c_{29} c_{8} t^{3} u{\left(t \right)} + c_{29} c_{8} t^{2} u{\left(t \right)} + 2 c_{29} c_{9} t^{3} + 2 c_{29} t&=0.\\
\end{align*}
Now, we see that these two equations entails finding the roots of two polynomials $P_1$ and $P_2$ that are composed of the following monomials:  

\begin{equation}
M=\left[\begin{matrix}1\\v{\left(t \right)}\\v^{2}{\left(t \right)}\\u{\left(t \right)}\\u{\left(t \right)} v{\left(t \right)}\\u^{2}{\left(t \right)}\\t\\t v{\left(t \right)}\\t u{\left(t \right)}\\t^{2}\end{matrix}\right].
\end{equation}
So it is a matter of extracting the coefficients in front of these monomials and then setting the corresponding equation to zero. 
What happens when the monomial is 1?
$$a c_{1} c_{10} u{\left(t \right)} v{\left(t \right)} + a c_{1} c_{11} u{\left(t \right)} v^{2}{\left(t \right)} + a c_{1} c_{12} u{\left(t \right)} v^{3}{\left(t \right)} + a c_{1} c_{13} u^{2}{\left(t \right)} v{\left(t \right)} + a c_{1} c_{14} u^{2}{\left(t \right)} v^{2}{\left(t \right)} + a c_{1} c_{15} u^{3}{\left(t \right)} v{\left(t \right)} + a c_{1} c_{16} t u{\left(t \right)} v{\left(t \right)} + a c_{1} c_{17} t u{\left(t \right)} v^{2}{\left(t \right)} + a c_{1} c_{18} t u^{2}{\left(t \right)} v{\left(t \right)} + a c_{1} c_{19} t^{2} u{\left(t \right)} v{\left(t \right)} + a c_{10} c_{4} u^{2}{\left(t \right)} v{\left(t \right)} + a c_{10} c_{7} t u{\left(t \right)} v{\left(t \right)} + a c_{11} c_{4} u^{2}{\left(t \right)} v^{2}{\left(t \right)} + a c_{11} c_{7} t u{\left(t \right)} v^{2}{\left(t \right)} + a c_{11} u{\left(t \right)} v{\left(t \right)} + a c_{12} c_{4} u^{2}{\left(t \right)} v^{3}{\left(t \right)} + a c_{12} c_{7} t u{\left(t \right)} v^{3}{\left(t \right)} + a c_{13} c_{4} u^{3}{\left(t \right)} v{\left(t \right)} + a c_{13} c_{7} t u^{2}{\left(t \right)} v{\left(t \right)} + a c_{14} c_{4} u^{3}{\left(t \right)} v^{2}{\left(t \right)} + a c_{14} c_{7} t u^{2}{\left(t \right)} v^{2}{\left(t \right)} + a c_{14} u^{2}{\left(t \right)} v{\left(t \right)} + a c_{15} c_{4} u^{4}{\left(t \right)} v{\left(t \right)} + a c_{15} c_{7} t u^{3}{\left(t \right)} v{\left(t \right)} + a c_{16} c_{4} t u^{2}{\left(t \right)} v{\left(t \right)} + a c_{16} c_{7} t^{2} u{\left(t \right)} v{\left(t \right)} + a c_{17} c_{4} t u^{2}{\left(t \right)} v^{2}{\left(t \right)} + a c_{17} c_{7} t^{2} u{\left(t \right)} v^{2}{\left(t \right)} + a c_{17} t u{\left(t \right)} v{\left(t \right)} + a c_{18} c_{4} t u^{3}{\left(t \right)} v{\left(t \right)} + a c_{18} c_{7} t^{2} u^{2}{\left(t \right)} v{\left(t \right)} + a c_{19} c_{4} t^{2} u^{2}{\left(t \right)} v{\left(t \right)} + a c_{19} c_{7} t^{3} u{\left(t \right)} v{\left(t \right)} + c_{10} c_{3} u{\left(t \right)} + c_{10} c_{4} u{\left(t \right)} v{\left(t \right)} + c_{10} c_{6} + c_{10} c_{7} v{\left(t \right)} + c_{10} c_{8} t u{\left(t \right)} + c_{10} c_{8} u{\left(t \right)} + c_{10} v{\left(t \right)} + c_{11} c_{3} u{\left(t \right)} v{\left(t \right)} + c_{11} c_{4} u{\left(t \right)} v^{2}{\left(t \right)} + c_{11} c_{6} v{\left(t \right)} + c_{11} c_{7} v^{2}{\left(t \right)} + c_{11} c_{8} t u{\left(t \right)} v{\left(t \right)} + c_{11} c_{8} u{\left(t \right)} v{\left(t \right)} + c_{11} v^{2}{\left(t \right)} + c_{12} c_{3} u{\left(t \right)} v^{2}{\left(t \right)} + c_{12} c_{4} u{\left(t \right)} v^{3}{\left(t \right)} + c_{12} c_{6} v^{2}{\left(t \right)} + c_{12} c_{7} v^{3}{\left(t \right)} + c_{12} c_{8} t u{\left(t \right)} v^{2}{\left(t \right)} + c_{12} c_{8} u{\left(t \right)} v^{2}{\left(t \right)} + c_{12} v^{3}{\left(t \right)} + c_{13} c_{3} u^{2}{\left(t \right)} + c_{13} c_{4} u^{2}{\left(t \right)} v{\left(t \right)} + c_{13} c_{6} u{\left(t \right)} + c_{13} c_{7} u{\left(t \right)} v{\left(t \right)} + c_{13} c_{8} t u^{2}{\left(t \right)} + c_{13} c_{8} u^{2}{\left(t \right)} + c_{14} c_{3} u^{2}{\left(t \right)} v{\left(t \right)} + c_{14} c_{4} u^{2}{\left(t \right)} v^{2}{\left(t \right)} + c_{14} c_{6} u{\left(t \right)} v{\left(t \right)} + c_{14} c_{7} u{\left(t \right)} v^{2}{\left(t \right)} + c_{14} c_{8} t u^{2}{\left(t \right)} v{\left(t \right)} + c_{14} c_{8} u^{2}{\left(t \right)} v{\left(t \right)} + c_{15} c_{3} u^{3}{\left(t \right)} + c_{15} c_{4} u^{3}{\left(t \right)} v{\left(t \right)} + c_{15} c_{6} u^{2}{\left(t \right)} + c_{15} c_{7} u^{2}{\left(t \right)} v{\left(t \right)} + c_{15} c_{8} t u^{3}{\left(t \right)} + c_{15} c_{8} u^{3}{\left(t \right)} + c_{15} u^{2}{\left(t \right)} + c_{16} c_{3} t u{\left(t \right)} + c_{16} c_{4} t u{\left(t \right)} v{\left(t \right)} + c_{16} c_{6} t + c_{16} c_{7} t v{\left(t \right)} + c_{16} c_{8} t^{2} u{\left(t \right)} + c_{16} c_{8} t u{\left(t \right)} + c_{16} t v{\left(t \right)} + c_{16} + c_{17} c_{3} t u{\left(t \right)} v{\left(t \right)} + c_{17} c_{4} t u{\left(t \right)} v^{2}{\left(t \right)} + c_{17} c_{6} t v{\left(t \right)} + c_{17} c_{7} t v^{2}{\left(t \right)} + c_{17} c_{8} t^{2} u{\left(t \right)} v{\left(t \right)} + c_{17} c_{8} t u{\left(t \right)} v{\left(t \right)} + c_{17} t v^{2}{\left(t \right)} + c_{17} v{\left(t \right)} + c_{18} c_{3} t u^{2}{\left(t \right)} + c_{18} c_{4} t u^{2}{\left(t \right)} v{\left(t \right)} + c_{18} c_{6} t u{\left(t \right)} + c_{18} c_{7} t u{\left(t \right)} v{\left(t \right)} + c_{18} c_{8} t^{2} u^{2}{\left(t \right)} + c_{18} c_{8} t u^{2}{\left(t \right)} + c_{18} u{\left(t \right)} + c_{19} c_{3} t^{2} u{\left(t \right)} + c_{19} c_{4} t^{2} u{\left(t \right)} v{\left(t \right)} + c_{19} c_{6} t^{2} + c_{19} c_{7} t^{2} v{\left(t \right)} + c_{19} c_{8} t^{3} u{\left(t \right)} + c_{19} c_{8} t^{2} u{\left(t \right)} + c_{19} t^{2} v{\left(t \right)} + c_{20} u{\left(t \right)} + c_{21} u{\left(t \right)} v{\left(t \right)} + c_{22} u{\left(t \right)} v^{2}{\left(t \right)} + c_{23} u^{2}{\left(t \right)} + c_{24} u^{2}{\left(t \right)} v{\left(t \right)} + c_{25} u^{3}{\left(t \right)} + c_{26} t u{\left(t \right)} + c_{27} t u{\left(t \right)} v{\left(t \right)} + c_{28} t u^{2}{\left(t \right)} + c_{29} t^{2} u{\left(t \right)} + u{\left(t \right)} v{\left(t \right)} e^{- c_{0}} e^{- c_{1} v{\left(t \right)}} e^{- c_{2} v^{2}{\left(t \right)}} e^{- c_{3} u{\left(t \right)}} e^{- c_{5} u^{2}{\left(t \right)}} e^{- c_{6} t} e^{- c_{9} t^{2}} e^{- c_{4} u{\left(t \right)} v{\left(t \right)}} e^{- c_{7} t v{\left(t \right)}} e^{- c_{8} t u{\left(t \right)}} \frac{d}{d t} \xi{\left(t \right)}$$
Only extracting the exponential bit?

$$\left(- a u{\left(t \right)} v{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)} + a v{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)}\right) e^{- c_{0}} e^{- c_{1} v{\left(t \right)}} e^{- c_{2} v^{2}{\left(t \right)}} e^{- c_{3} u{\left(t \right)}} e^{- c_{5} u^{2}{\left(t \right)}} e^{- c_{6} t} e^{- c_{9} t^{2}} e^{- c_{4} u{\left(t \right)} v{\left(t \right)}} e^{- c_{7} t v{\left(t \right)}} e^{- c_{8} t u{\left(t \right)}}$$


### Looping over these linearised symmetry conditions and extracting the coefficients

In [9]:
# Allocate memory for the equations that we want to solve
ansatze_equations = []
# Loop over the linearised symmetry conditions
for lin_sym_index in range(len(lin_syms_tidy)):
    # Extract the coefficient in front of the exponential term
    coeff_temp = lin_syms_tidy[lin_sym_index].coeff(expand(1/exp(polynomial_list[0])))
    # Make sure that none of the variables occur in the equations at hand
    #for state in states:
        #coeff_temp = coeff_temp.subs(state,0)
    #coeff_temp = coeff_temp.subs(t,0)        
    # Append our triplet
    ansatze_equations.append((lin_sym_index,expand(1/exp(polynomial_list[0])),coeff_temp))
    # Loop over the monomials and re-do this exercise for each monomial
    for monomial in M:
        coeff_temp = expand(lin_syms_tidy[lin_sym_index].coeff(monomial) - (lin_syms_tidy[lin_sym_index].coeff(monomial).coeff(expand(1/exp(polynomial_list[0])))*expand(1/exp(polynomial_list[0]))))           
        #if monomial == 1:
            #coeff_temp = expand(lin_syms_tidy[lin_sym_index].coeff(monomial) - (lin_syms_tidy[lin_sym_index].coeff(expand(1/exp(polynomial_list[0])))*expand(1/exp(polynomial_list[0]))))           
        #else:
            # Extract the coefficient in front of the exponential term
            #coeff_temp = lin_syms_tidy[lin_sym_index].coeff(monomial)
        # Make sure that none of the variables occur in the equations at hand
        for state in states:
            coeff_temp = coeff_temp.subs(state,0)
        coeff_temp = coeff_temp.subs(t,0)                    
        # Append our triplet
        ansatze_equations.append((lin_sym_index,monomial,coeff_temp))   
# Now we are ready to print our lovely equations 
for eq_index in range(len(ansatze_equations)):
    # Begin the equations environment
    if eq_index == 0:
        print("Equations stemming from linearised symmetry condition %d\n"%(ansatze_equations[eq_index][0]+1))
        print("\\begin{align*}")        
    elif eq_index < len(ansatze_equations)-1 and ansatze_equations[eq_index][0]>ansatze_equations[eq_index-1][0]:
        print("Equations stemming from linearised symmetry condition %d\n"%(ansatze_equations[eq_index][0]+1))
        print("\\begin{align*}")
    # Otherwise we just print the equations
    print("%s:&%s&=0,\\\\"%(latex(ansatze_equations[eq_index][1]),latex(ansatze_equations[eq_index][2])))
    # End the equations environment
    if eq_index < len(ansatze_equations)-1 and ansatze_equations[eq_index][0]<ansatze_equations[eq_index+1][0]:
        print("\\end{align*}")        
    elif eq_index == len(ansatze_equations)-1:
        print("\\end{align*}")        
            
    

Equations stemming from linearised symmetry condition 1

\begin{align*}
e^{- c_{0}} e^{- c_{1} v{\left(t \right)}} e^{- c_{2} v^{2}{\left(t \right)}} e^{- c_{3} u{\left(t \right)}} e^{- c_{5} u^{2}{\left(t \right)}} e^{- c_{6} t} e^{- c_{9} t^{2}} e^{- c_{4} u{\left(t \right)} v{\left(t \right)}} e^{- c_{7} t v{\left(t \right)}} e^{- c_{8} t u{\left(t \right)}}:&u{\left(t \right)} v{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)} - u{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)}&=0,\\
1:&c_{10} c_{6} + c_{16}&=0,\\
v{\left(t \right)}:&- a c_{1} c_{10} - a c_{11} + c_{10} c_{7} + c_{10} + c_{11} c_{6} - c_{11} + c_{17}&=0,\\
v^{2}{\left(t \right)}:&- a c_{1} c_{11} - 2 a c_{10} c_{2} - 2 a c_{12} + c_{11} c_{7} + c_{11} + c_{12} c_{6} - c_{12}&=0,\\
u{\left(t \right)}:&c_{10} c_{3} + c_{10} c_{8} + c_{13} c_{6} + c_{18} + c_{20}&=0,\\
u{\left(t \right)} v{\left(t \right)}:&a c_{1} c_{10} - a c_{1} c_{13} - a c_{10} c_{4} + a c_{11} - a c_{14} - c_{10} c_{3} + c_{10} c_{4} + c_

Equations stemming from linearised symmetry condition 1

\begin{align*}
e^{- c_{0}} e^{- c_{1} v{\left(t \right)}} e^{- c_{2} v^{2}{\left(t \right)}} e^{- c_{3} u{\left(t \right)}} e^{- c_{5} u^{2}{\left(t \right)}} e^{- c_{6} t} e^{- c_{9} t^{2}} e^{- c_{4} u{\left(t \right)} v{\left(t \right)}} e^{- c_{7} t v{\left(t \right)}} e^{- c_{8} t u{\left(t \right)}}:&u{\left(t \right)} v{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)} - u{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)}&=0,\\
1:&c_{10} c_{6} + c_{16}&=0,\\
v{\left(t \right)}:&- a c_{1} c_{10} - a c_{11} + c_{10} c_{7} + c_{10} + c_{11} c_{6} - c_{11} + c_{17}&=0,\\
v^{2}{\left(t \right)}:&- a c_{1} c_{11} - 2 a c_{10} c_{2} - 2 a c_{12} + c_{11} c_{7} + c_{11} + c_{12} c_{6} - c_{12}&=0,\\
u{\left(t \right)}:&c_{10} c_{3} + c_{10} c_{8} + c_{13} c_{6} + c_{18} + c_{20}&=0,\\
u{\left(t \right)} v{\left(t \right)}:&a c_{1} c_{10} - a c_{1} c_{13} - a c_{10} c_{4} + a c_{11} - a c_{14} - c_{10} c_{3} + c_{10} c_{4} + c_{11} c_{3} + c_{11} c_{8} + c_{13} c_{7} + c_{14} c_{6} + c_{21}&=0,\\
u^{2}{\left(t \right)}:&2 c_{10} c_{5} + c_{13} c_{3} + c_{13} c_{8} + c_{15} c_{6} + c_{15} + c_{23}&=0,\\
t:&2 c_{10} c_{9} + c_{16} c_{6} - c_{16} + 2 c_{19}&=0,\\
t v{\left(t \right)}:&- a c_{1} c_{16} - a c_{10} c_{7} - a c_{17} + 2 c_{11} c_{9} + c_{16} c_{7} + c_{16} + c_{17} c_{6} - c_{17}&=0,\\
t u{\left(t \right)}:&c_{10} c_{8} + 2 c_{13} c_{9} + c_{16} c_{3} + c_{16} c_{8} + c_{18} c_{6} + c_{26}&=0,\\
t^{2}:&2 c_{16} c_{9} + c_{19} c_{6} - c_{19}&=0,\\
\end{align*}
Equations stemming from linearised symmetry condition 2

\begin{align*}
e^{- c_{0}} e^{- c_{1} v{\left(t \right)}} e^{- c_{2} v^{2}{\left(t \right)}} e^{- c_{3} u{\left(t \right)}} e^{- c_{5} u^{2}{\left(t \right)}} e^{- c_{6} t} e^{- c_{9} t^{2}} e^{- c_{4} u{\left(t \right)} v{\left(t \right)}} e^{- c_{7} t v{\left(t \right)}} e^{- c_{8} t u{\left(t \right)}}:&- a u{\left(t \right)} v{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)} + a v{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)}&=0,\\
1:&a c_{20} + c_{20} c_{6} + c_{26}&=0,\\
v{\left(t \right)}:&- a c_{1} c_{20} - a c_{10} + c_{20} c_{7} + c_{21} c_{6} + c_{27}&=0,\\
v^{2}{\left(t \right)}:&- a c_{1} c_{21} - a c_{11} - 2 a c_{2} c_{20} - a c_{22} + c_{21} c_{7} + c_{22} c_{6}&=0,\\
u{\left(t \right)}:&- a c_{20} + a c_{23} + c_{20} c_{3} + c_{20} c_{8} + c_{23} c_{6} + c_{23} + c_{28}&=0,\\
u{\left(t \right)} v{\left(t \right)}:&a c_{1} c_{20} - a c_{1} c_{23} - a c_{13} - a c_{20} c_{4} - c_{20} c_{3} + c_{20} c_{4} + c_{21} c_{3} + c_{21} c_{8} + c_{23} c_{7} - c_{23} + c_{24} c_{6} + c_{24}&=0,\\
u^{2}{\left(t \right)}:&- a c_{23} + a c_{25} + 2 c_{20} c_{5} + c_{23} c_{3} + c_{23} c_{8} + c_{25} c_{6} + 2 c_{25}&=0,\\
t:&a c_{26} + 2 c_{20} c_{9} + c_{26} c_{6} + 2 c_{29}&=0,\\
t v{\left(t \right)}:&- a c_{1} c_{26} - a c_{16} - a c_{20} c_{7} + 2 c_{21} c_{9} + c_{26} c_{7} + c_{27} c_{6}&=0,\\
t u{\left(t \right)}:&- a c_{26} + a c_{28} + c_{20} c_{8} + 2 c_{23} c_{9} + c_{26} c_{3} + c_{26} c_{8} + c_{28} c_{6} + c_{28}&=0,\\
t^{2}:&a c_{29} + 2 c_{26} c_{9} + c_{29} c_{6}&=0,\\
\end{align*}

# Solving the remaining equations
The last step now is to solve the equations above. I guess we can use a "greedy approach" where we just loop through each equation one at a time, solve for one of the coefficients, then we substitute the solution into the coefficient vector as well as into all other equations. What can happen then is that certain equations become "redundant" if you will, so we have to remove them from the lists of equation, or not consider them. How to solve this system is what I would like to discuss tomorrow!

In [11]:
print(len(c))
print(c)
print(len(c_copy))
print(c_copy)

30
[c_0, c_1, c_2, c_3, c_4, c_5, c_6, c_7, c_8, c_9, c_10, c_11, c_12, c_13, c_14, c_15, c_16, c_17, c_18, c_19, c_20, c_21, c_22, c_23, c_24, c_25, c_26, c_27, c_28, c_29]
30
[c_0, (-a*c_10*c_11 + c_10**2*(c_7 + 1) - c_10*(c_11 - c_17) - c_11*c_16)/(a*c_10**2), c_2, c_3, c_4, c_5, -c_16/c_10, c_7, c_8, c_9, c_10, c_11, c_12, c_13, c_14, c_15, c_16, c_17, c_18, c_19, c_20, c_21, c_22, c_23, c_24, c_25, c_26, c_27, c_28, c_29]


In [17]:
# Sort the equations 
c_copy, eqs_coeffs, eqs_coeffs_no_consts = solve_algebraic_equations(c,ansatze_equations)
# Make a copy of the tangents as well
tangent_copy = []
for eta in eta_list:
    tangent_copy.append(expand(eta))
for tangent_index in range(len(tangent_copy)):
    for sub_index in range(len(c)):
        tangent_copy[tangent_index] = tangent_copy[tangent_index].subs(c[sub_index],c_copy[sub_index])

print(simplify(tangent_copy[0]-eta_list[0]))
print(simplify(tangent_copy[1]-eta_list[1]))
print(simplify(tangent_copy[2]-eta_list[2]))
#tangent_copy = [tangent.subs(c[sub_index],c_copy[sub_index]) for tangent in tangent_list for sub_index in range(len(c))]
#print(tangent_copy)
# PRINTING ALL OF THIS TO A FILE!
original_stdout = sys.stdout # Save a reference to the original standard output
with open('LV_output_polynomial_times_exponential_ansatze.tex', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    # Print a minimal preamble to a latex document
    print("\\documentclass{article}")
    print("% General document formatting")
    print("\\usepackage[margin=0.7in]{geometry}")
    print("\\usepackage[parfill]{parskip}")
    print("\\usepackage[utf8]{inputenc}\n")
    print("% Related to math")
    print("\\usepackage{amsmath,amssymb,amsfonts,amsthm}\n")
    print("\\begin{document}\n")
    # Print some things where we set up the document
    print("We're studying the Lotka-Volterra model:")
    print("\\begin{align*}")
    print("\\dfrac{\mathrm{d}u}{\mathrm{d}t}&=%s,\\\\"%(latex(omega_list[0])))
    print("\\dfrac{\mathrm{d}v}{\mathrm{d}t}&=%s.\\\\"%(latex(omega_list[1])))
    print("\\end{align*}")
    print("We want to find an infinitesimal generator of the Lie group of the following kind:")
    print("$$X=\\xi(t)\\partial t+\\eta_{1}(t,u,v)\\partial_u+\\eta_{2}\\partial_v.$$")
    # Prompt to the user
    print("The infinitesimals or the tangents in this generator solves the following equations:")
    print(lin_sym_original_str)
    print("We use the following ansatze for the infinitesimals or the tangents:")
    print("\\begin{align*}")
    print("\\eta_1&=P_1\\exp\\left(P_0\\right),\\")
    print("\\eta_2&=P_2\\exp\\left(P_0\\right).\\")
    print("\\end{align*}")
    # Print the degree of the polynomials and the unknown coefficients
    print("We use ansatze of the type $$\\eta_i=P_{i}\\exp\\left(P_{0}\\right)$$ for some index $i\\in\\left\\{1,\\ldots,n\\right\\}$ where $n$ is the number of states.")
    print("Here, our two polynomials $P_{1}$ and $P_{2}$ are multivariate polynomials of a given degree. In this case, the degree of our polynomials are: $%d$."%(degree_polynomial))
    print("Moreover, we have the following states or dependent variables:\n%s\nand one independent variable $t$."%(latex(Matrix(len(states),1,states),mode='equation').replace("\\begin{equation}","\\begin{equation}\n").replace("\\end{equation}","\n\\end{equation}")))
    print("Now, given all variables (independent and dependent) as well as the degree of the polynomials, we obtain the following monomials:")
    print(latex(Matrix(len(M),1,M),mode='equation').replace("\\begin{equation}","\\begin{equation}\nM=").replace("\\end{equation}",".\n\\end{equation}"))
    print("Furthermore, we have the following unknown coefficients:")
    print(latex(Matrix(len(c),1,c),mode='equation').replace("\\begin{equation}","\\begin{equation}\n\\mathbf{c}=").replace("\\end{equation}",".\n\\end{equation}"))
    print("Now, given all of these unknowns and coefficient, this is the ansatze we get:")
    print("\\begin{align*}")
    for eta_index in range(len(eta_list)):
        temp_str = "\\eta_{%d}&=" + latex(eta_list[eta_index]) 
        if eta_index == len(eta_list)-1:
            temp_str += ".\\\\"
        else:
            temp_str += ",\\\\"        
        print(temp_str % (eta_index+1))
    print("\\end{align*}")
    print("After plugging in these enormous ansatze, we organise these equations in terms of the various monomials (or linearly independent basis functions).")
    # Now we are ready to print our lovely equations 
    print("Equations without coefficients in them:")
    print("\\begin{align*}")        
    for eq_index in range(len(eqs_coeffs_no_consts)):
        # Otherwise we just print the equations
        print("%s:&%s&=0,\\\\"%(latex(eqs_coeffs_no_consts[eq_index][1]),latex(eqs_coeffs_no_consts[eq_index][2])))
    # End align environment
    print("\\end{align*}")      
    # Now we are ready to print our lovely equations 
    print("Equations with coefficients in them:")
    print("\\begin{align*}")        
    for eq_index in range(len(eqs_coeffs)):
        # Otherwise we just print the equations
        print("%s:&%s&=0,\\\\"%(latex(eqs_coeffs[eq_index][1]),latex(eqs_coeffs[eq_index][2])))
    # End align environment
    print("\\end{align*}") 
    # Print the solved coefficients as well
    print("The coefficients we calculated were:")
    print(latex(Matrix(len(c_copy),1,c_copy),mode='equation').replace("\\begin{equation}","\\begin{equation}\n\\mathbf{c}=").replace("\\end{equation}",".\n\\end{equation}"))
    # Print the tangents as well
    print("The tangents we calculated were:")
    print("\\begin{align*}")
    for tangent_index in range(len(tangent_copy)):
        if tangent_index  == 0:
            print("\\xi&=%s,\\\\"%(latex(tangent_copy[tangent_index])))
        elif tangent_index == (len(tangent_copy)-1):
            print("\\eta_{%d}&=%s.\\\\"%(tangent_index,latex(tangent_copy[tangent_index])))
        else:
            print("\\eta_{%d}&=%s,\\\\"%(tangent_index,latex(tangent_copy[tangent_index])))                  
    print("\\end{align*}\n")
    print("\\end{document}")
    # End printing to file
    sys.stdout = original_stdout # Reset the standard output to its original value

Equation:	c_10*c_6 + c_16	=	0
Solution:	c_6	=	-c_16/c_10
Equation:	-a*c_1*c_10 - a*c_11 + c_10*c_7 + c_10 - c_11 + c_17 - c_11*c_16/c_10	=	0
Solution:	c_1	=	(-a*c_10*c_11 + c_10**2*(c_7 + 1) + c_10*(-c_11 + c_17) - c_11*c_16)/(a*c_10**2)
0
0
0


Equations without coefficients in them:
\begin{align*}
e^{- c_{0}} e^{- c_{1} v{\left(t \right)}} e^{- c_{2} v^{2}{\left(t \right)}} e^{- c_{3} u{\left(t \right)}} e^{- c_{5} u^{2}{\left(t \right)}} e^{- c_{6} t} e^{- c_{9} t^{2}} e^{- c_{4} u{\left(t \right)} v{\left(t \right)}} e^{- c_{7} t v{\left(t \right)}} e^{- c_{8} t u{\left(t \right)}}:&u{\left(t \right)} v{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)} - u{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)}&=0,\\
e^{- c_{0}} e^{- c_{1} v{\left(t \right)}} e^{- c_{2} v^{2}{\left(t \right)}} e^{- c_{3} u{\left(t \right)}} e^{- c_{5} u^{2}{\left(t \right)}} e^{- c_{6} t} e^{- c_{9} t^{2}} e^{- c_{4} u{\left(t \right)} v{\left(t \right)}} e^{- c_{7} t v{\left(t \right)}} e^{- c_{8} t u{\left(t \right)}}:&- a u{\left(t \right)} v{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)} + a v{\left(t \right)} \frac{d}{d t} \xi{\left(t \right)}&=0,\\
\end{align*}
Equations with coefficients in them:
\begin{align*}
1:&c_{10} c_{6} + c_{16}&=0,\\
v{\left(t \right)}:&- a c_{1} c_{10} - a c_{11} + c_{10} c_{7} + c_{10} + c_{11} c_{6} - c_{11} + c_{17}&=0,\\
v^{2}{\left(t \right)}:&- a c_{1} c_{11} - 2 a c_{10} c_{2} - 2 a c_{12} + c_{11} c_{7} + c_{11} + c_{12} c_{6} - c_{12}&=0,\\
u{\left(t \right)}:&c_{10} c_{3} + c_{10} c_{8} + c_{13} c_{6} + c_{18} + c_{20}&=0,\\
u{\left(t \right)} v{\left(t \right)}:&a c_{1} c_{10} - a c_{1} c_{13} - a c_{10} c_{4} + a c_{11} - a c_{14} - c_{10} c_{3} + c_{10} c_{4} + c_{11} c_{3} + c_{11} c_{8} + c_{13} c_{7} + c_{14} c_{6} + c_{21}&=0,\\
u^{2}{\left(t \right)}:&2 c_{10} c_{5} + c_{13} c_{3} + c_{13} c_{8} + c_{15} c_{6} + c_{15} + c_{23}&=0,\\
t:&2 c_{10} c_{9} + c_{16} c_{6} - c_{16} + 2 c_{19}&=0,\\
t v{\left(t \right)}:&- a c_{1} c_{16} - a c_{10} c_{7} - a c_{17} + 2 c_{11} c_{9} + c_{16} c_{7} + c_{16} + c_{17} c_{6} - c_{17}&=0,\\
t u{\left(t \right)}:&c_{10} c_{8} + 2 c_{13} c_{9} + c_{16} c_{3} + c_{16} c_{8} + c_{18} c_{6} + c_{26}&=0,\\
t^{2}:&2 c_{16} c_{9} + c_{19} c_{6} - c_{19}&=0,\\
1:&a c_{20} + c_{20} c_{6} + c_{26}&=0,\\
v{\left(t \right)}:&- a c_{1} c_{20} - a c_{10} + c_{20} c_{7} + c_{21} c_{6} + c_{27}&=0,\\
v^{2}{\left(t \right)}:&- a c_{1} c_{21} - a c_{11} - 2 a c_{2} c_{20} - a c_{22} + c_{21} c_{7} + c_{22} c_{6}&=0,\\
u{\left(t \right)}:&- a c_{20} + a c_{23} + c_{20} c_{3} + c_{20} c_{8} + c_{23} c_{6} + c_{23} + c_{28}&=0,\\
u{\left(t \right)} v{\left(t \right)}:&a c_{1} c_{20} - a c_{1} c_{23} - a c_{13} - a c_{20} c_{4} - c_{20} c_{3} + c_{20} c_{4} + c_{21} c_{3} + c_{21} c_{8} + c_{23} c_{7} - c_{23} + c_{24} c_{6} + c_{24}&=0,\\
u^{2}{\left(t \right)}:&- a c_{23} + a c_{25} + 2 c_{20} c_{5} + c_{23} c_{3} + c_{23} c_{8} + c_{25} c_{6} + 2 c_{25}&=0,\\
t:&a c_{26} + 2 c_{20} c_{9} + c_{26} c_{6} + 2 c_{29}&=0,\\
t v{\left(t \right)}:&- a c_{1} c_{26} - a c_{16} - a c_{20} c_{7} + 2 c_{21} c_{9} + c_{26} c_{7} + c_{27} c_{6}&=0,\\
t u{\left(t \right)}:&- a c_{26} + a c_{28} + c_{20} c_{8} + 2 c_{23} c_{9} + c_{26} c_{3} + c_{26} c_{8} + c_{28} c_{6} + c_{28}&=0,\\
t^{2}:&a c_{29} + 2 c_{26} c_{9} + c_{29} c_{6}&=0,\\
\end{align*}

